In [2]:
import requests
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [ ]:
json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

In [4]:
access_token = authenticate(json['client_id'], json['client_secret'])

In [ ]:
bigquery.run_query()

In [ ]:
catalog_item_id = 'MLB28887155'

url = f"https://api.mercadolibre.com/products/{catalog_item_id}/items"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

In [ ]:
item_id = 'MLB4966133390'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

In [8]:
item_id = 'MLB4978023790'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
data = response.json()

In [ ]:
def process_shipping(json_data):
    try:
        default_value = json_data.get('default')
        channels = json_data.get('channels', [])
        item_id = json_data.get('item_id')
        dict_list = []
        for channel in channels:
            dict_content = {
                'item_id': item_id,
                'channel_id': channel.get('id'),
                'mode': channel.get('mode'),
                'logistic_type': channel.get('logistic_type'),
                'local_pick_up': channel.get('local_pick_up'),
                'free_shipping': channel.get('free_shipping'),
                'store_pick_up': channel.get('store_pick_up'),
                'default': default_value
            }
            dict_list.append(dict_content)
        return dict_list
    except Exception as e:
        print(f'Error processing json: {json_data}, error: {e}')
        return []
import pandas as pd
pd.DataFrame(process_shipping(data))

In [ ]:
data = json
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_FULLFILMENT
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/catelog_details/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names='inventory_id'
)

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json


def insert_bq_competitors_prices(request):

    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_CATALOG_COMPETITORS
    blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')

    df_processed_data = pd.DataFrame()

    for date in list_dates_to_process:

        # Transform date to string
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')
        # Get blob with the date
        blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
        # List all the files
        blobs = storage.list_blobs(bucket_name, blob_prefix)

        # Processing each blob
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)

            for json in content:
                processed_dict = process_prices(json)

                if isinstance(processed_dict, list):
                    df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
                else:
                    continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)
        
        print('** Correct dataframe schema **')
        bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ**')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)


def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]

                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')
        

In [2]:
json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [ ]:
data = json
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_CATALOG_COMPETITORS
blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        for json in content:
            processed_dict = process_competitors_catalog(json)
            if isinstance(processed_dict, list):
                df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
            else:
                continue
    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id

In [ ]:
df_processed_data

In [ ]:
content[0]['item_id']

In [ ]:
data = content[0]['results']
data[0]

In [ ]:
content[0]['results'][0].get('category_id')

In [ ]:
list_proc = process_competitors_catalog(content[0])
pd.DataFrame(list_proc)

In [28]:
def process_competitors_catalog(json):

    catalog_id = json['item_id']
    results_list = []  # Create an empty list to store the dictionaries

    try:
        for item in json['results']:
            dict_content = {
                'catalog_product_id': catalog_id, 
                'item_id' : item.get('item_id'),
                'competitors_type': 'catalog',
                'category_id': item.get('category_id'),
                'official_store_id': item.get('official_store_id'),
                'competitor_seller_id': item.get('seller_id'),
                'listing_type_id': item.get('listing_type_id'),
                'condition': item.get('condition'),
            }
            
            results_list.append(dict_content)  # Append each dictionary to the list
        
        return results_list  # Return the full list after iterating through all items
    
    except Exception as e:
        print(f'Error processing json: {json}. Error: {str(e)}')
        return None  # Optionally return None if there's an error


In [2]:
# Improve costs
data = json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [3]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import requests

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests


# Parsing request data
# data = request.get_json()
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COSTS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Getting params to see costs
query = f'''
    with items_details as (
    select distinct
        item_id,
        listing_type,
        category_id
    from datalake-v2-424516.datalake_v2.items_details
    where
        1=1
        and date(correspondent_date) = current_date()
        and seller_id = {seller_id}
    )
    select 
    p.item_id as id,
    d.listing_type as listing_type_id,
    d.category_id,
    p.price,
    p.channel
    from datalake-v2-424516.datalake_v2.items_prices p
    inner join items_details d
    on p.item_id = d.item_id
    where 
        1=1
        and date(p.correspondent_date) = current_date()
        and channel is not null
'''
# blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
# items_id = fetch_items_from_storage(
# storage, 
# bucket_name, 
# blob_items_prefix, 
# key_names=['id','price', 'category_id', 'listing_type_id']
# )

df_params = bigquery.run_query(query)
# items = df_params[['id','channel']].to_dict(orient='records')
df_params['channel'] = df_params['channel'].apply(lambda x : x.replace('channel_', '')).drop(columns = 'channel')
items_id = df_params.to_dict(orient='records')

print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving 
blob_basic_path = settings.BLOB_COSTS(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_COST
headers = {'Authorization': f'Bearer {access_token}'}

# Batch processing the API requests
async with aiohttp.ClientSession() as session:
    await batch_process(session, items_id, url, headers, 
                        bucket_name, date_blob_path, storage, 
                        params = items_id, add_item_id = True)
    
log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)



** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
** Items found: 1976**
** Cleaning blob **
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_0__process_time=2024-10-15T14:29:29.211919-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_10__process_time=2024-10-15T14:30:30.258868-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_11__process_time=2024-10-15T14:30:30.304502-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_12__process_time=2024-10-15T14:30:30.257919-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_13__process_time=2024-10-15T14:30:30.346773-0

In [4]:
log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)
